In [1]:
import utility
from web_scraper import get_comments
import pandas as pd
import numpy
import datetime
from monkeylearn import MonkeyLearn
ml = MonkeyLearn('f4bbf60d362e38d5897af58b0500519959783001')
model_id = "cl_pi3C7JiL"

keywords = {"msft": ["msft", "microsoft", "gates"], "tsla": ["tsla", "tesla", "musk"],
            "aapl": ["aapl", "apple", "cook"], "fb": ["fb", "facebook", "zuckerberg"],
            "amzn": ["amzn", "amazon", "bezos"], "googl": ["googl", "google", "alphabet", "pichai"]}

settings_file = utility.read_settings("settings.txt")
[company, start_date, end_date, subreddits] = settings_file

In [2]:
comments = pd.DataFrame()
for single_date in utility.daterange(start_date, end_date):
    for subreddit in subreddits:
        print("Checking " + str(subreddit) + "on" + str(single_date))
        new_data = get_comments(subreddit, single_date, single_date+datetime.timedelta(days=1), keywords[company])
        comments = comments.append(new_data)

comments.to_csv(str(company) + "_one_week_data.csv")

Checking algotradingon2020-10-11
Checking dividendson2020-10-11
Checking etoroon2020-10-11
Checking financeon2020-10-11
Checking forexon2020-10-11
Checking investingon2020-10-11
Checking optionson2020-10-11
Checking optionsexclusiveon2020-10-11
Checking personalfinanceon2020-10-11
Checking robinhoodon2020-10-11
Checking securityanalysison2020-10-11
Checking stockmarketon2020-10-11
Checking stockson2020-10-11
Checking tdameritradeon2020-10-11
Checking trading212on2020-10-11
Checking wallstreetbetson2020-10-11
Checking webullon2020-10-11
Checking algotradingon2020-10-12
Checking dividendson2020-10-12
Checking etoroon2020-10-12
Checking financeon2020-10-12
Checking forexon2020-10-12
Checking investingon2020-10-12
Checking optionson2020-10-12
Checking optionsexclusiveon2020-10-12
Checking personalfinanceon2020-10-12
Checking robinhoodon2020-10-12
Checking securityanalysison2020-10-12
Checking stockmarketon2020-10-12
Checking stockson2020-10-12
Checking tdameritradeon2020-10-12
Checking tra

In [3]:
company_data = comments.reset_index()
company_data = company_data.drop(columns=["index"])
data = company_data["Body"].values.tolist()
response = ml.classifiers.classify(model_id, data)

In [4]:
tags, confidences = utility.extract_classification(response.body)
company_data["Tag"], company_data["Classifier"] = tags, confidences

In [5]:
result = company_data.copy()
result = result.drop(columns=["ID", "Author", "Body"])
print(result.head())

         Date  Subreddit       Tag  Classifier
0  2020-10-11  dividends   Neutral       0.644
1  2020-10-11      Forex  Negative       0.836
2  2020-10-11      Forex  Negative       0.990
3  2020-10-11      Forex  Negative       0.942
4  2020-10-11  investing  Negative       0.917


In [6]:
from dictionary_sentiment_analysis import vectorized_sentiment_check
result["Dictionary_Tag"] = vectorized_sentiment_check(company_data["Body"])
print(result.head())

         Date  Subreddit       Tag  Classifier  Dictionary_Tag
0  2020-10-11  dividends   Neutral       0.644             0.0
1  2020-10-11      Forex  Negative       0.836             0.0
2  2020-10-11      Forex  Negative       0.990             0.5
3  2020-10-11      Forex  Negative       0.942             0.0
4  2020-10-11  investing  Negative       0.917             0.5


In [7]:
result.to_csv("amzn_result_one_week.csv")